In [1]:
import re
import csv
from getpass import getpass
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver import Chrome
import datetime as dt
from pytz import timezone
import pymongo
from pymongo import MongoClient
from hashlib import md5

In [2]:
#password = "rootcmgnews"
cluster = MongoClient("mongodb://127.0.0.1:27017/")
db = cluster["Scrappy2DB-localtest"]
Tcollection = db["warehouse"]
tweetHash = db["warehash"]

In [3]:
hcol = tweetHash.find({})
h5list = [i["_id"] for i in hcol]

In [5]:
def convert_dt(z_time, tz):
     assert z_time[-1] == 'Z'
     z_time = z_time[:-1] + '000'
     z_time_dt = dt.datetime.strptime(z_time, '%Y-%m-%dT%H:%M:%S.%f')
     my_dt = z_time_dt.replace(tzinfo=timezone('EST')).astimezone(tz)
     date = my_dt.strftime("%Y-%m-%d")
     time = my_dt.strftime("%H:%M:%S")
     return date, time

def get_tweet_data(card):
    username = card.find_element_by_xpath('.//span').text
    handle = card.find_element_by_xpath('.//span[contains(text(), "@")]').text
    try:
        postDate = card.find_element_by_xpath('.//time').get_attribute('datetime')
        date, time = convert_dt(postDate, timezone('EST'))
    except NoSuchElementException:
        return
    text = card.find_element_by_xpath('.//div[2]/div[2]/div[2]').text
    reply_cnt = card.find_element_by_xpath('.//div[@data-testid="reply"]').text
    retweet_cnt = card.find_element_by_xpath('.//div[@data-testid="retweet"]').text
    like_cnt = card.find_element_by_xpath('.//div[@data-testid="like"]').text
    
    tweet = (username, handle, date, time, text, reply_cnt, retweet_cnt, like_cnt)
    return tweet


PATH = 'C:\Program Files (x86)\chromedriver.exe'
driver = Chrome(PATH)

def twitterpages(url,acc,cName):
    driver.get(url)
    driver.maximize_window()
    sleep(2)

    #driver.find_element_by_link_text('Tweets').click()
    tNow = dt.datetime.now()
    idtime = tNow.strftime('%y%m%d%H')
    run = tNow.strftime('%d/%m/%Y - %H:%M')
    pcount = 0

    tweet_ids = set()
    last_position = driver.execute_script("return window.pageYOffset;") 
    scrolling = True

    while scrolling:
        page_cards = driver.find_elements_by_xpath('//*[@data-testid="tweet"]')
        for card in page_cards[-15:]:       
            tweet = get_tweet_data(card)
            if tweet:
                tweet_id = ''.join(tweet)
                if tweet_id not in tweet_ids:
                    tweet_ids.add(tweet_id)
                   

                    t_id = f"{idtime}{acc}-{pcount}"
                    h5 = md5(f'{tweet[4]}-{tweet[2]}{acc}'.encode()).hexdigest()
                    if h5 in h5list:
                        continue
                    tweetHash.insert_one({"_id":h5,"date":run})
                    h5list.append(h5)
                    
                    try:
                        Tcollection.insert_one({
                            '_id' : t_id, 
                            'category' : 'Twitter', 
                            'sub-cat':cName,
                            'Lang' : 'English',
                            'Run' : run, 
                            'Title':tweet[4],
                            'Date':tweet[2],
                            'Time':tweet[3], 
                            'Comments':tweet[5],
                            'Likes':tweet[6],
                            'Retweets':tweet[7]
                            })
                    except:
                        pass
                    pcount += 1
                else:
                    return None 
                
        scroll_attempt = 0
        while True:
            driver.execute_script('window.scrollTo(0, 22800);')
            sleep(1)
            curr_position = driver.execute_script("return window.pageYOffset;")
            if last_position == curr_position:
                scroll_attempt += 1

                # end of scroll region
                if scroll_attempt >= 3:
                    scrolling = False
                    break
                else:
                    sleep(2) # attempt another scroll
            else:
                last_position = curr_position
                break
                
t_pgs = [("https://twitter.com/newsfirstsl?lang=en" ,"N", "News First"), ("https://twitter.com/adaderana", "D", "Derana News"),("https://twitter.com/HiruNewsEnglish" ,"H", "Hiru News")]

for i in t_pgs:
    twitterpages(i[0],i[1],i[2])
    
driver.close()    

In [ ]:
#document.body.scrollHeight

In [ ]:
#driver.close()

In [7]:
# Tdel = Tcollection.delete_many({"_id":{ '$regex': '' }})
# print(f'Documents deleted: {Tdel.deleted_count}')

Documents deleted: 566


In [8]:
# Tdel = tweetHash.delete_many({"_id":{ '$regex': '' }})
# print(f'Documents deleted: {Tdel.deleted_count}')

Documents deleted: 566


In [ ]:
#     comment = card.find_element_by_xpath('//*[@id="react-root"]/div/div/div[2]/main/div/div/div/div[1]/div/div[2]/div/div/div[2]/section/div/div/div[1]/div/div/article/div/div/div/div[2]/div[2]/div[2]/div[1]//div[1]/span[1]')
#     #text = [i.text for i in comment][0]
#     #print(text)
#     try:
#         text = [i.text for i in comment]
#         print(text, handle)
#     except:
#          text = comment.text
         #print(text)
    #responding = card.find_element_by_xpath('.//div[2]/div[2]/div[2]').text
    #text = comment + responding

In [10]:
# col = Tcollection.find({})

In [11]:
# for i in col:
#     dtime = i['Date']
#     if dtime == None:
#         continue
#     try:
#         print(dtime)
#         dnt = dt.datetime.strptime(dtime, '%d/%m/%Y')
#         dnt = dnt.strftime('%Y-%m-%d')
#         print(dnt)
#         print(i['_id'])
#         Tcollection.update_one({"_id": i['_id']}, {"$set": {'Date':dnt}})
#     except:
#         pass

21/09/2021
2021-09-21
2200N-0
21/09/2021
2021-09-21
2200N-1
21/09/2021
2021-09-21
2200N-2
21/09/2021
2021-09-21
2200N-3
21/09/2021
2021-09-21
2200N-4
21/09/2021
2021-09-21
2200N-5
21/09/2021
2021-09-21
2200N-6
21/09/2021
2021-09-21
2200N-7
21/09/2021
2021-09-21
2200N-8
21/09/2021
2021-09-21
2200N-9
21/09/2021
2021-09-21
2200N-10
20/09/2021
2021-09-20
2200N-11
20/09/2021
2021-09-20
2200N-12
20/09/2021
2021-09-20
2200N-13
20/09/2021
2021-09-20
2200N-14
20/09/2021
2021-09-20
2200N-15
20/09/2021
2021-09-20
2200N-16
20/09/2021
2021-09-20
2200N-17
20/09/2021
2021-09-20
2200N-18
20/09/2021
2021-09-20
2200N-19
20/09/2021
2021-09-20
2200N-20
20/09/2021
2021-09-20
2200N-21
20/09/2021
2021-09-20
2200N-22
19/09/2021
2021-09-19
2200N-23
19/09/2021
2021-09-19
2200N-24
19/09/2021
2021-09-19
2200N-25
19/09/2021
2021-09-19
2200N-26
19/09/2021
2021-09-19
2200N-27
19/09/2021
2021-09-19
2200N-28
19/09/2021
2021-09-19
2200N-29
19/09/2021
2021-09-19
2200N-30
19/09/2021
2021-09-19
2200N-31
19/09/2021
2021-09

29/08/2021
2021-08-29
2200N-260
29/08/2021
2021-08-29
2200N-261
29/08/2021
2021-08-29
2200N-262
29/08/2021
2021-08-29
2200N-263
29/08/2021
2021-08-29
2200N-264
29/08/2021
2021-08-29
2200N-265
29/08/2021
2021-08-29
2200N-266
29/08/2021
2021-08-29
2200N-267
29/08/2021
2021-08-29
2200N-268
29/08/2021
2021-08-29
2200N-269
29/08/2021
2021-08-29
2200N-270
29/08/2021
2021-08-29
2200N-271
29/08/2021
2021-08-29
2200N-272
29/08/2021
2021-08-29
2200N-273
28/08/2021
2021-08-28
2200N-274
28/08/2021
2021-08-28
2200N-275
28/08/2021
2021-08-28
2200N-276
28/08/2021
2021-08-28
2200N-277
28/08/2021
2021-08-28
2200N-278
28/08/2021
2021-08-28
2200N-279
28/08/2021
2021-08-28
2200N-280
28/08/2021
2021-08-28
2200N-281
27/08/2021
2021-08-27
2200N-282
27/08/2021
2021-08-27
2200N-283
27/08/2021
2021-08-27
2200N-284
27/08/2021
2021-08-27
2200N-285
27/08/2021
2021-08-27
2200N-286
27/08/2021
2021-08-27
2200N-287
27/08/2021
2021-08-27
2200N-288
27/08/2021
2021-08-27
2200N-289
27/08/2021
2021-08-27
2200N-290
26/08/20

07/08/2021
2021-08-07
2200N-517
07/08/2021
2021-08-07
2200N-518
07/08/2021
2021-08-07
2200N-519
07/08/2021
2021-08-07
2200N-520
07/08/2021
2021-08-07
2200N-521
07/08/2021
2021-08-07
2200N-522
07/08/2021
2021-08-07
2200N-523
07/08/2021
2021-08-07
2200N-524
07/08/2021
2021-08-07
2200N-525
06/08/2021
2021-08-06
2200N-526
06/08/2021
2021-08-06
2200N-527
06/08/2021
2021-08-06
2200N-528
06/08/2021
2021-08-06
2200N-529
06/08/2021
2021-08-06
2200N-530
06/08/2021
2021-08-06
2200N-531
06/08/2021
2021-08-06
2200N-532
06/08/2021
2021-08-06
2200N-533
06/08/2021
2021-08-06
2200N-534
06/08/2021
2021-08-06
2200N-535
06/08/2021
2021-08-06
2200N-536
06/08/2021
2021-08-06
2200N-537
06/08/2021
2021-08-06
2200N-538
21/09/2021
2021-09-21
2201D-0
21/09/2021
2021-09-21
2201D-1
21/09/2021
2021-09-21
2201D-2
21/09/2021
2021-09-21
2201D-3
21/09/2021
2021-09-21
2201D-4
21/09/2021
2021-09-21
2201D-5
21/09/2021
2021-09-21
2201D-6
21/09/2021
2021-09-21
2201D-7
21/09/2021
2021-09-21
2201D-8
21/09/2021
2021-09-21
2201

10/09/2021
2021-09-10
2201D-238
10/09/2021
2021-09-10
2201D-239
10/09/2021
2021-09-10
2201D-240
10/09/2021
2021-09-10
2201D-241
09/09/2021
2021-09-09
2201D-242
09/09/2021
2021-09-09
2201D-243
09/09/2021
2021-09-09
2201D-244
09/09/2021
2021-09-09
2201D-245
09/09/2021
2021-09-09
2201D-246
09/09/2021
2021-09-09
2201D-247
09/09/2021
2021-09-09
2201D-248
09/09/2021
2021-09-09
2201D-249
09/09/2021
2021-09-09
2201D-250
09/09/2021
2021-09-09
2201D-251
09/09/2021
2021-09-09
2201D-252
09/09/2021
2021-09-09
2201D-253
09/09/2021
2021-09-09
2201D-254
09/09/2021
2021-09-09
2201D-255
09/09/2021
2021-09-09
2201D-256
09/09/2021
2021-09-09
2201D-257
09/09/2021
2021-09-09
2201D-258
09/09/2021
2021-09-09
2201D-259
09/09/2021
2021-09-09
2201D-260
09/09/2021
2021-09-09
2201D-261
09/09/2021
2021-09-09
2201D-262
09/09/2021
2021-09-09
2201D-263
09/09/2021
2021-09-09
2201D-264
09/09/2021
2021-09-09
2201D-265
09/09/2021
2021-09-09
2201D-266
09/09/2021
2021-09-09
2201D-267
09/09/2021
2021-09-09
2201D-268
08/09/20

21/09/2021
2021-09-21
2201H-8
21/09/2021
2021-09-21
2201H-9
21/09/2021
2021-09-21
2201H-10
21/09/2021
2021-09-21
2201H-11
21/09/2021
2021-09-21
2201H-12
21/09/2021
2021-09-21
2201H-13
21/09/2021
2021-09-21
2201H-14
21/09/2021
2021-09-21
2201H-15
21/09/2021
2021-09-21
2201H-16
20/09/2021
2021-09-20
2201H-17
20/09/2021
2021-09-20
2201H-18
20/09/2021
2021-09-20
2201H-19
20/09/2021
2021-09-20
2201H-20
20/09/2021
2021-09-20
2201H-21
20/09/2021
2021-09-20
2201H-22
20/09/2021
2021-09-20
2201H-23
20/09/2021
2021-09-20
2201H-24
20/09/2021
2021-09-20
2201H-25
20/09/2021
2021-09-20
2201H-26
20/09/2021
2021-09-20
2201H-27
20/09/2021
2021-09-20
2201H-28
20/09/2021
2021-09-20
2201H-29
20/09/2021
2021-09-20
2201H-30
20/09/2021
2021-09-20
2201H-31
20/09/2021
2021-09-20
2201H-32
20/09/2021
2021-09-20
2201H-33
18/09/2021
2021-09-18
2201H-34
18/09/2021
2021-09-18
2201H-35
18/09/2021
2021-09-18
2201H-36
18/09/2021
2021-09-18
2201H-37
18/09/2021
2021-09-18
2201H-38
18/09/2021
2021-09-18
2201H-39
18/09/2021

04/09/2021
2021-09-04
2201H-267
04/09/2021
2021-09-04
2201H-268
04/09/2021
2021-09-04
2201H-269
04/09/2021
2021-09-04
2201H-270
04/09/2021
2021-09-04
2201H-271
04/09/2021
2021-09-04
2201H-272
04/09/2021
2021-09-04
2201H-273
04/09/2021
2021-09-04
2201H-274
03/09/2021
2021-09-03
2201H-275
03/09/2021
2021-09-03
2201H-276
03/09/2021
2021-09-03
2201H-277
03/09/2021
2021-09-03
2201H-278
03/09/2021
2021-09-03
2201H-279
03/09/2021
2021-09-03
2201H-280
03/09/2021
2021-09-03
2201H-281
03/09/2021
2021-09-03
2201H-282
03/09/2021
2021-09-03
2201H-283
03/09/2021
2021-09-03
2201H-284
03/09/2021
2021-09-03
2201H-285
03/09/2021
2021-09-03
2201H-286
03/09/2021
2021-09-03
2201H-287
03/09/2021
2021-09-03
2201H-288
03/09/2021
2021-09-03
2201H-289
03/09/2021
2021-09-03
2201H-290
03/09/2021
2021-09-03
2201H-291
03/09/2021
2021-09-03
2201H-292
03/09/2021
2021-09-03
2201H-293
03/09/2021
2021-09-03
2201H-294
02/09/2021
2021-09-02
2201H-295
02/09/2021
2021-09-02
2201H-296
02/09/2021
2021-09-02
2201H-297
02/09/20